In [ ]:
import sys
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
import pandas as pd
import numpy as np
import glob, os

import itertools

import statsmodels.tsa.stattools as ts

from backtester.exoinfo import EXOInfo
from exobuilder.data.exostorage import EXOStorage
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np
import scipy
import pprint

%pylab inline
figsize(20,10)

In [ ]:
def hurst(ts):
    """Returns the Hurst Exponent of the time series vector ts"""
    # Create the range of lag values
    lags = range(2, 20)
    # Calculate the array of the variances of the lagged differences
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]
    # Use a linear fit to estimate the Hurst Exponent
    poly = polyfit(log(lags), log(tau), 1)
    # Return the Hurst exponent from the polyfit output
    return poly[0]*2.0

In [ ]:
# Only Cont Fut Flag
only_cont_fut = False

In [ ]:
# Loading global setting for MongoDB etc.
from scripts.settings import *

try:
    from scripts.settings_local import *
except:
    pass

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)
pp = pprint.PrettyPrinter(indent=4)

exo_list = storage.exo_list()
pp.pprint(exo_list)


df = pd.DataFrame()

for e in exo_list:
    exo_series, exo_dict = storage.load_series(e)
    if only_cont_fut == True:   
        if 'Fut' in exo_dict['name']:   
            exo_series = exo_series[exo_series.columns[0]]
            df[exo_dict['name']] = exo_series
            
    if only_cont_fut == False:
        exo_series = exo_series[exo_series.columns[0]]
        df[exo_dict['name']] = exo_series
#exo_series.plot()

In [ ]:
df.head()

In [ ]:
px_ser = df.ES_SMART_Ichimoku_Straddle_Bullish

In [ ]:
hurst_stats = pd.Series(0.0, index=px_ser.index)

window_start = 0
for i in range(0, px_ser.size, 30):
    hurst_stats.iloc[window_start : i] = hurst(px_ser.iloc[window_start : i])
    window_start = i

In [ ]:
px_ser.dropna().plot()
hurst_stats.plot(secondary_y=True)

In [ ]:
#rolling_adfteststats = px_ser.dropna().rolling(50).apply(lambda x: ts.adfuller(x)[0])
rolling_hurst = px_ser.dropna().rolling(40).apply(lambda x: hurst(x))

In [ ]:
px_ser.dropna().ix[:'2016'].plot()
rolling_hurst.ix[:'2016'].plot(secondary_y=True)

#rolling_adfteststats.ix[:'2015'].plot(secondary_y=True)
#rstats.plot(secondary_y=True)
axhline(rolling_hurst.mean())
#axhline(-1)

In [ ]:
ts.adfuller(rolling_hurst.dropna())

In [ ]:
rolling_hurst.rank(pct=True).plot()

In [ ]:
rolling_hurst_pctrank = rolling_hurst.rank(pct=True)

after_high_ser = pd.Series(0.0, index=px_ser.index, name='hurst after high pct rank value')
after_low_ser = pd.Series(0.0, index=px_ser.index, name='hurst after low pct rank value')

for i in range(px_ser.size):
    if rolling_hurst_pctrank[i] <= 0.1:
        #ser.ix[ser.index[i]: ser.index[i] + pd.DateOffset(10)] = px_ser.diff().ix[ser.index[i]: ser.index[i] + pd.DateOffset(10)]
        
        after_low_ser.ix[px_ser.index[i]: px_ser.index[i] + pd.DateOffset(10)] = px_ser.rolling(10).std().ix[px_ser.index[i]: 
                                                                                            px_ser.index[i] + pd.DateOffset(10)]
    
    
    if rolling_hurst_pctrank[i] >= 0.9:
        #ser.ix[ser.index[i]: ser.index[i] + pd.DateOffset(10)] = px_ser.diff().ix[ser.index[i]: ser.index[i] + pd.DateOffset(10)]
        
        after_high_ser.ix[px_ser.index[i]: px_ser.index[i] + pd.DateOffset(10)] = px_ser.rolling(10).std().ix[px_ser.index[i]: 
                                                                                            px_ser.index[i] + pd.DateOffset(10)]

In [ ]:
after_low_ser.dropna().plot()


after_high_ser.dropna().plot()

axhline(after_high_ser.dropna().mean())

axhline(after_low_ser.dropna().mean())

In [ ]:
#px_ser.diff().dropna().plot.kde()

px_ser.rolling(10).std().dropna().plot.kde()
#after_low_ser.dropna().plot.kde()
after_high_ser.dropna().plot.kde()

In [ ]:
px_ser.diff().dropna().plot.kde()

px_ser.diff().dropna().ix['2015-12'].plot.kde()

In [ ]:
ser.cumsum().plot()

In [ ]:
rolling_hurst_pctrank.ix[ rolling_hurst_pctrank.index[80]:
                         rolling_hurst_pctrank.index[80] + pd.DateOffset(10)]

In [ ]:
 rolling_hurst_pctrank.index[8] + pd.DateOffset(10)